**3장의 simple word2vec에 두 가지 개선 방안 추가**

1. Embedding이라는 새로운 계층 도입

2. 네거티브 샘플링이라는 새로운 손실 함수 도입

# word2vec 개선 (1)

3장에서의 CBOW 모델은 다루는 어휘의 수가 총 7개이기에 문제없이 작동한다. 하지만 거대한 말뭉치를 다루게 되면 문제가 발생한다.  

어휘가 100만 개, 은닉층의 뉴런이 100개인 CBOW를 생각해보자. 입력층과 출력층에는 각 100만 개의 뉴런이 존재한다.

#### 이 때 다음의 두 가지 병목현상이 발생한다.


- **입력층의 원핫 표현과 가중치 행렬 $W_{in}$ 의 곱 계산**
  - 원핫 표현과 관련된 문제
  - `Embedding`이라는 새로운 계층 도입
  
        
- **은닉층과 가중치 행렬 $W_{out}$의 곱 및 Softmax 계층의 계산**
  - 은닉층 이후의 계산 문제
  - 은닉층과 가중치 행렬 $W_{out}$의 곱의 계산량이 상당함
  - `negative sampling` 이라는 새로운 손실 함수 도입
---

## Embedding 계층

어휘의 수가 100만 개, 은닉층의 뉴런이 100개인 경우를 가정해보자.

$c * W_{in} = h$ 일 때, 형상은 다음과 같다.

- c.shape = (1, 1000000)
- W.shape = (1000000, 100)
- h.shape = (1,100)

**> 이는 결과적으로 $W_{in}$ 행렬에서 특정 행을 추출하는 것 뿐이다.**   
여기서 말하는 "특정 행"이라 함은 $W_{in}$ 행렬에서 input c 단어의 id에 해당하는 인덱스 행을 말함

### 즉, 원핫 표현과 MatMul 계층의 행렬곱 연산은 사실 상 불필요한 작업임!

그러면 가중치 매개변수로부터 `단어 ID에 해당하는 행(벡터)`를 추출하는 계층을 만들어보자!<br>
<u>여기서의 계층이 바로 Embedding 계층!</u>

**참고) Embedding은 단어 임베딩이라는 용어에서 유래했다.

즉, 우리가 할 일은 Embedding 계층에 단어 임베딩 (분산 표현)을 저장하는 것이다.

`Q?` 기존의 W에 저장될 분산 표현이 Embedding 계층으로 옮겨간 것?

## Embedding 계층 구현

In [1]:
import numpy as np

W = np.arange(21).reshape(7,3)
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [2]:
# 단일 행 뽑기
# idx = 2
display(W[2])
# idx = 5
display(W[5])

array([6, 7, 8])

array([15, 16, 17])

In [3]:
# idx 여러개 한번에 추출
idx = np.array([1,0,3,0])
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

In [4]:
class Embedding: 
    def __init__(self,W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None # 단어 ID 인덱스를 배열로 저장, 미니배치를 고려할 때 여러개도 추출될 수 있도록 구현
        
    # 가중치 W의 특정 행을 추출
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx] 
        return out
    
    # 반대로 역전파에서는 반대로 출력층에서의 기울기를 W의 idx 행에 할당
    def backward(self, dout):
        dW, = self.grads # 가중치 기울기 dW를 꺼낸다
        dW[...] = 0 # dW의 형상을 유지한 채 원소만 0으로 덮어쓴다
        
        # idx 중복을 고려하지 않은 방법
        # dW[self.idx] = dout # 앞 층에서 전해진 기울기 dout을 idx 번째 행에 할당한다. 
        
        # idx 중복 고려 : 할당 x 더하기 !
        # Method #1 느리당
        #for i, word_id in enumerate(self.idx):
        #    dW[word_id] += dout[i]
        
        # Method #2 효율적. np.add.at(A,idx,B) = B를 A의 idx번째 행에 더해준다.
        np.add.at(dW, self.idx, dout)
        
        return None

**[주의]** 역전파 과정에서 idx의 중복을 고려하여 값을 '할당'하는 것이 아니라 값을 **더하기** 한다.

### 단점
 - 다의어 문제 해결 불가
 - 단어의 위치 고려가 되지 않음
 
**-> 궁극적으로 문장 단위로 가야 하는 이유 (Elmo, Transformer)**

**Embedding 계층 구현으로 인한 이점**

: 입력 측 MatMul 계층을 Embedding 계층으로 전환하여 메모리 사용량을 줄이고 쓸데없는 계산량을 생략!

# word2vec 개선 (2)

은닉층 이후, <u>행렬 곱과 Softmax 계층의 계산</u>에서 발생하는 병목 현상을 해결해보자

by **`네거티브 샘플링 기법`**

Softmax 대신 네거티브 샘플링을 이용하면 어휘가 아무리 많아져도 계산량을 억제할 수 있다.

어렵다! 복잡하다!

## 은닉층 이후 계산의 문제점

마찬가지로 어휘가 100만 개, 은닉층 뉴런의 수가 100개인 CBOW 모델을 가정해보자

<img src="../imgs/fig 4-2.png" width="500" align='left'>
<br></br>

**문제점 1. 은닉층의 뉴런 X $W_{out}$**

    - 은닉층 벡터의 크기가 100
    - 가중치 행렬의 크기는 100 X 100만
<br>

**문제점 2. Softmax 계층의 계산**
    
    - Softmax의 계산식에서 분모의 값을 얻기 위해 Exponential 계산을 100만 번 수행해야 한다...!
    

## 다중 분류에서 이진 분류로

네거티브 샘플링 기법의 핵심 아이디어는 `이진 분류 (binary classification)` 에 있다.   

즉, **<u>다중 분류를 이진 분류로 근사하는 것</u>**

    simple_CBOW 에서의 방식 : 100만 개의 단어 중 옳은 단어 하나를 선택하는 문제   

        you, goodbye ---> 타깃단어는 무엇인가? ---> say (1/100만)

    네거티브 샘플링 방식 : 100만 개의 단어 중 특정 단어가 옳은 단어인지 아닌지 이진 판단하는 문제

        you, goodbye ---> say인가? (Yes, No)

본 방법을 구현하기 위해서는 ?
#### > 출력층에 뉴런을 하나만 준비하면 된다. 출력층의 뉴런이 `say`의 점수를 출력하는 것이다. 

`이전에는 출력층에 softmax로 모든 단어에 대한 확률을 출력한 뒤, 가장 높은 확률의 단어를 예측하는 식으로 계산을 수행했다면,    
이제는 출력층의 say에 해당하는 단어에 sigmoid 함수를 적용시켜 say에 해당하는 확률값만을 받아온다.
모든 단어에 대한 계산을 단일 단어에 대한 계산으로 변환`

## 시그모이드 함수와 교차 엔트로피 오차

sigmoid 함수 : 

$ y = \frac{1}{1 + exp(-x)} $

교차 엔트로피 오차 : 

$L = -(tlogy + (1-t)log(1-y))$

 - y : 시그모이드 함수의 출력
 - t : 정답 레이블 (1 혹은 0)
 - t = 1  > 정답 > -logy
 - t = 0 > 정답 아님 > -log(1-y)
 
**Sigmoid + Cross Entropy Error를 조합하여 역전파의 값이 y-t 라는 값으로 깔끔하게 도출된다!**

## 다중 분류에서 이진 분류로 (구현)

Embed -> hidden -> Embed dot -> Sigmoid with loss

** Embedding Dot Layer : Embedding Layer + dot product

In [5]:
class EmbeddingDot:
    def __init__(self, W):
        # 총 4개의 인스턴스 변수
        self.embed = Embedding(W) # Embedding 계층
        self.params = self.embed.params # 매개변수 저장
        self.grads = self.embed.grads # 기울기 저장
        self.cache = None # 순전파 시의 계산 결과를 잠시 유지하기 위해 사용되는 변수 
        
    # 순전파 메서드에서는 은닉층 뉴런과 단어 ID의 넘파이 배열(미니배치)을 받는다.
    def forward(self, h, idx):
        target_W = self.embed.forward(idx) # embedding 계층의 forward(idx)를 호출하여 idx에 해당하는 행 추출
        out = np.sum(target_W * h, axis = 1) # 내적 계산 이후 행마다 더하여 최종결과 out 반환
        
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0],1)
        
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

In [6]:
W = np.arange(21).reshape(7,3)
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [7]:
idx = [0,3,1]
target_W = W[idx]
target_W

array([[ 0,  1,  2],
       [ 9, 10, 11],
       [ 3,  4,  5]])

In [8]:
h = np.arange(9).reshape(3,3)
h

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [9]:
target_W * h

array([[ 0,  1,  4],
       [27, 40, 55],
       [18, 28, 40]])

In [10]:
np.sum(target_W * h, axis = 1)

array([  5, 122,  86])

## 네거티브 샘플링

현재까지는 정답에 대해서만 학습.    
예를 들어, `you`와 `goodbye` 가 맥락 입력으로 들어갔을 때 정답 레이블이 `say`인 경우, 좋은 가중치가 준비되어 있다면 Sigmoid 계층의 확률은 1에 가까울 것이다

**But 오답 (say 이외의 단어)을 입력하면 어떤 결과가 나올지에 대해서도 처리해줘야함!**

우리의 목적 :
1. 긍정적 예 ("say")에 대해서는 Sigmoid 의 출력이 1에 가깝게 
2. 부정적 예 ("say" 외 단어)에 대해서는 Sigmoid의 출력이 0에 가깝게

`어떻게 2번을 구현할 수 있을까?`

모든 부정적 예를 대상으로 이진 분류 학습 ? 
**No** 이전과 마찬가지로 high cost!

**근사적인 해답으로 부정적 예를 몇 개 선택한다. 즉, 적은 수의 부정적 예를 샘플링해 사용한다. (Negative Sampling)**

---
### Summary
1. 네거티브 샘플링 기법은 긍정적 예를 타깃으로 한 경우의 손실을 구한다. 
2. 동시에 부정적 예를 몇 개 샘플링(선별)하여 각 부정적 예에 대하여 손실을 구한다. 
3. (1 + 2) 각 손실을 더한 값을 최종 손실로 정한다.

### Example

1. say ... 1
2. hello ... 0 K ... 0
3. sum(Loss)

## 네거티브 샘플링의 샘플링 기법

**그렇다면 부정적 예를 어떻게 샘플링 하는가?**

무작위 샘플링? 

    No! 희소한 단어만 샘플링되었다면 결과가 나빠질 것

말뭉치의 통계 데이터를 기초로 샘플링해보자. 

**말뭉치에서 자주 등장하는 단어를 많이 추출하고 드물게 등장하는 단어를 적게 추출하자**


In [11]:
import numpy as np

# 0-9 숫자 중 하나 무작위 샘플링
np.random.choice(10)

6

In [12]:
# 단어에서도 마찬가지
words = ['you', 'say', 'goodbye' ,'I', 'hello', '.']
np.random.choice(words)

'goodbye'

In [13]:
# 중복 허용 추출
print(np.random.choice(words, size = 5))
# 5개만 무작위 샘플링 (중복 없음)
print(np.random.choice(words, size = 5, replace = False))
# 확률분포에 따라 샘플링
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
print(np.random.choice(words, p=p)) # 인수 p에 확률분포 리스트를 지정하면 확률분포대로 샘플링한다!

['.' 'hello' 'you' '.' 'goodbye']
['.' 'you' 'goodbye' 'say' 'I']
I


한편, word2vec의 네거티브 샘플링에서는 앞의 확률분포 (p)에 0.75를 제곱하라고 권고한다.

**why?**  <u>0.75 제곱을 함으로써 출현 확률이 낮은 단어를 버리지 않기 위해</u>

**0.75 라는 수치에 대한 이론적 근거는 없다**

In [14]:
p = [0.7, 0.29, 0.01]
new_p = np.power(p, 0.75)
new_p /= np.sum(new_p)
print(new_p)

[0.64196878 0.33150408 0.02652714]


0.75 제곱을 통해 낮은 확률이 약간 상승함

In [15]:
class UnigramSampler:
    # 초기화 시에 3개의 인수를 받는다
    # 단어 ID 목록, 확률분포에 제곱할 값, 부정적 예시 샘플링할 개수
    def __init__(self, corpus, power, sample_size): 
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        # 단어 빈도 산출
        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1
        
        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]
        
        # 단어의 빈도 기준 확률 분포 산출
        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)
        
    
    # target으로 지정한 단어를 긍정적 예로 해석하고, 그 외의 단어 ID를 샘플링
    def get_negative_sample(self, target):
        batch_size = target.shape[0]
        GPU = False
        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
        else:
            # GPU(cupy）로 계산할 때는 속도를 우선한다.
            # 부정적 예에 타깃이 포함될 수 있다.
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample

### 활용 예시

In [17]:
import collections

In [18]:
corpus = np.array([0, 1, 2, 3, 4, 1, 2, 3])
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1,3,0])
negative_sample = sampler.get_negative_sample(target)
print(negative_sample)

[[0 2]
 [0 1]
 [4 1]]


- target 1에 해당하는 부정적 예시 2 0
- target 3에 해당하는 부정적 예시 샘플링 1 0
- target 0에 해당하는 부정적 예시 샘플링 1 3

## 네거티브 샘플링 구현

In [19]:
class NegativeSamplingLoss:
    # 출력 가중치 W, 말뭉치 ID 리스트, 확률분포에 제곱할 값, 샘플링 횟수
    def __init__(self, W, corpus, power = 0.75, sample_size = 5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)

        # 원하는 계층을 리스트로 보관
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)] # 부정적 예시(sample_size) + 긍정적 예시 (1)
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]
        
        self.params, self.grads = [],[]
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads
    
    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target) # 부정적 예를 샘플링하여 변수에 저장
        
        #긍정적 예 순전파. 0번째 계층
        score = self.embed_dot_layers[0].forward(h,target)
        correct_label = np.ones(batch_size, dtype=np.int32) # 1
        loss = self.loss_layers[0].forward(score, correct_label)
        
        #부정적 예 순전파 
        negative_label = np.zeros(batch_size, dtype=np.int32) # 0
        for i in range(self.sample_size):
            negative_target = negative_sample[:,i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target) 
            loss += self.loss_layers[1 + i].forward(score, negative_label)

        return loss
    
    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)
            
        return dh

# 개선판 word2vec 학습

## CBOW 모델 구현

In [50]:
class CBOW:
    # 어휘수, 은닉층 뉴런수, 맥락 크기, 단어 ID 목록
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f') # embedding layer를 사용하므로 둘의 형상이 같다.

        # 계층 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # Embedding 계층 2*window_size만큼 사용
            self.in_layers.append(layer) # 배열로 보관
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        # 모든 가중치와 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target) # loss가 다르다! 간결해짐
        return loss

    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

## CBOW 모델 학습 코드

In [1]:
import sys
sys.path.append("..")
import numpy as np
from common_wj import config
# GPU에서 실행하려면 아래 주석을 해제하세요(CuPy 필요).
# ===============================================
# config.GPU = True
# ===============================================
import pickle
from common_wj.trainer import Trainer
from common_wj.optimizer import Adam
from cbow import CBOW
from common_wj.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb

In [2]:
# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)

if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

# 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
# model = SkipGram(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'  # or 'skipgram_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)

| epoch 1 |  iter 1 / 9295 | time 0[s] | loss 4.16
| epoch 1 |  iter 21 / 9295 | time 1[s] | loss 4.16
| epoch 1 |  iter 41 / 9295 | time 2[s] | loss 4.15
| epoch 1 |  iter 61 / 9295 | time 3[s] | loss 4.12
| epoch 1 |  iter 81 / 9295 | time 4[s] | loss 4.05
| epoch 1 |  iter 101 / 9295 | time 5[s] | loss 3.92
| epoch 1 |  iter 121 / 9295 | time 6[s] | loss 3.78
| epoch 1 |  iter 141 / 9295 | time 7[s] | loss 3.63
| epoch 1 |  iter 161 / 9295 | time 8[s] | loss 3.49


KeyboardInterrupt: 

# Word analogy test

    Syntactic:
    e.g., bad : worst = good : best
    Semantics:
    e.g., Seoul : Korea = Tokyo : Japan

**Why does this imply that LMs are well trained? What does it have to do with vector arithmetic? When does this hold?**

--> word2vec을 수학적으로 접근해서 성능을 분석하는 논문이 2019년 ACL에 발표되었다.

#### Ethayarajh et al., Towards Understanding Linear Word Analogies, ACL 2019

word analogy 를 함수로 접근!

Analogy 𝑓 가 set of ordered pairs 𝑆 상의 invertible transformation이다


seoul, Korea -> 한국 `수도` = 서울 / function = "의 수도"!

https://kawine.github.io/blog/nlp/2019/06/21/word-analogies.html

<img src="../imgs/word_analogy.png" width="1000" align='left'>